In [1]:
# Python 2
import cv2
import numpy as np
import pandas as pd
import pickle
import random
from __future__ import print_function
from os import listdir
from os.path import isfile, join
from scipy.spatial.distance import cosine as cos_dist
from scipy.spatial.distance import euclidean as euclid_dist
from sklearn.neighbors import KNeighborsClassifier as kNN

In [2]:
def init_df(img_dir, num_sift_kp):
    df = pd.DataFrame(filter(lambda x: ".JPG" in x, listdir(img_dir)), columns=["name"])
    df["class"] = np.repeat(np.linspace(0, 49, num=50), 4)
    df["class"] = df["class"].astype("category")
    key_points, kp_descriptors = list(), list()
    df_name = list(df["name"])
    sift = cv2.SIFT(nfeatures=num_sift_kp)  # each image may produce a few more sift features
    for i in range(0, 200):
        progress = int(i / 200.0 * 1000) / 10.0
        print("{} {}%".format(df_name[i], progress), end="\r")
        img = cv2.imread(join(img_dir, df_name[i]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(img, None)
        if len(kp) > num_sift_kp:
            kp = kp[:num_sift_kp]
            des = des[:num_sift_kp]
        key_points.append(map(lambda x: {"pt": x.pt, "angle": x.angle, "size": x.size,
                                         "octave": x.octave, "response": x.response}, kp))
        kp_descriptors.append(des.astype(np.uint16))
    df["sift_key_points"] = key_points
    df["sift_kp_descriptors"] = kp_descriptors
    return df

In [3]:
df_cache = "df.pickle"
if isfile(df_cache):
    with open(df_cache, mode="rb") as h:
        df = pickle.load(h)
else:
    df = init_df("data", num_sift_kp=20)
    with open(df_cache, mode="wb") as h:
        pickle.dump(df, h, protocol=2)
print(df.shape)
print(df.dtypes)
print(df.head())

(200, 4)
name                     object
class                  category
sift_key_points          object
sift_kp_descriptors      object
dtype: object
           name class                                    sift_key_points  \
0  image001.JPG     0  [{u'pt': (124.926483154, 144.085632324), u'ang...   
1  image002.JPG     0  [{u'pt': (156.000778198, 182.117752075), u'ang...   
2  image003.JPG     0  [{u'pt': (112.154243469, 122.479042053), u'ang...   
3  image004.JPG     0  [{u'pt': (191.072982788, 121.957305908), u'ang...   
4  image006.JPG     1  [{u'pt': (112.855133057, 91.5557556152), u'ang...   

                                 sift_kp_descriptors  
0  [[3, 121, 65, 3, 7, 3, 5, 3, 4, 41, 54, 39, 12...  
1  [[7, 19, 8, 1, 1, 0, 16, 16, 6, 16, 20, 11, 1,...  
2  [[138, 2, 0, 0, 55, 7, 0, 3, 53, 10, 0, 0, 142...  
3  [[10, 42, 7, 2, 84, 81, 6, 0, 74, 109, 3, 2, 8...  
4  [[100, 7, 0, 0, 22, 35, 44, 53, 61, 5, 0, 0, 1...  


In [4]:
def new_split():
    split = ["train"] * 200
    for i in range(0, 50):
        split[i * 4 + random.randint(0, 3)] = "test"
    return split
for i in range(0, 10):
    split_name = "split_" + str(i)
    df[split_name] = new_split()
    df[split_name] = df[split_name].astype("category")
print(df.shape)
print(df.head())

(200, 14)
           name class                                    sift_key_points  \
0  image001.JPG     0  [{u'pt': (124.926483154, 144.085632324), u'ang...   
1  image002.JPG     0  [{u'pt': (156.000778198, 182.117752075), u'ang...   
2  image003.JPG     0  [{u'pt': (112.154243469, 122.479042053), u'ang...   
3  image004.JPG     0  [{u'pt': (191.072982788, 121.957305908), u'ang...   
4  image006.JPG     1  [{u'pt': (112.855133057, 91.5557556152), u'ang...   

                                 sift_kp_descriptors split_0 split_1 split_2  \
0  [[3, 121, 65, 3, 7, 3, 5, 3, 4, 41, 54, 39, 12...   train    test    test   
1  [[7, 19, 8, 1, 1, 0, 16, 16, 6, 16, 20, 11, 1,...   train   train   train   
2  [[138, 2, 0, 0, 55, 7, 0, 3, 53, 10, 0, 0, 142...   train   train   train   
3  [[10, 42, 7, 2, 84, 81, 6, 0, 74, 109, 3, 2, 8...    test   train   train   
4  [[100, 7, 0, 0, 22, 35, 44, 53, 61, 5, 0, 0, 1...   train    test   train   

  split_3 split_4 split_5 split_6 split_7 split_8 sp

In [5]:
class sNN:
    # C := # of classes; N := # of data point per training class; D := dimension of data point
    def __init__(self, distance_func):
        self.dist_func = distance_func  # (R^D, R^D) -> R

    def fit(self, train_x):  # C * N * D
        self.train_x = train_x
        return self

    def predict(self, test_x):  # R^D
        C = train_x.shape[0]
        dists = np.zeros(C, dtype=np.float32)
        for i in range(0, C):
            dists[i] = min([self.dist_func(test_x, x) for x in self.train_x[i]])
        return dists

In [6]:
def get_training_set(split_id):
    df_train = df[df["split_" + str(i)] == "train"]
    df_train_sift_des = list(df_train["sift_kp_descriptors"])
    return np.array([np.vstack(df_train_sift_des[j*3:j*3+3]) for j in range(0, 50)])  # 50 * 60 * 128

In [7]:
def test(classifier, split_id):
    df_test = df[df["split_" + str(split_id)] == "test"]
    test_x = np.array(df_test["sift_kp_descriptors"])  # 50 * 20 * 128
    predict_y = np.empty(50, dtype=np.uint8)
    for i in range(0, 50):
        dists = np.zeros(50, dtype=np.float32)
        for j in range(0, 20):
            dists += classifier.predict(test_x[i][j])
            progress = (float(i) * 20 + j) / 10
            print("split_{} {}%".format(split_id, progress), end="\r")
        predict_y[i] = np.argmin(dists)
    return predict_y

In [8]:
classifier = sNN(distance_func=euclid_dist)
tp_count = np.empty(10, dtype=np.uint8)
for i in range(0, 10):
    train_x = get_training_set(i)
    predict_y = test(classifier.fit(train_x), i)
    tp_count[i] = sum(np.linspace(0, 49, num=50) == predict_y)
print(tp_count)
accuracy = tp_count.astype(np.float32) / 50
print("mean_accuracy={}, sigma={}".format(np.mean(accuracy), np.std(accuracy)))

[45 46 44 46 43 46 45 46 47 47]
mean_accuracy=0.909999966621, sigma=0.0240831915289


In [9]:
# Naive kNN
def get_training_set(split_id):
    df_train = df[df["split_" + str(split_id)] == "train"]
    df_train_sift_des = list(df_train["sift_kp_descriptors"])
    df_train_class = list(df_train["class"])
    train_x, train_y = list(), list()
    for i in range(0, 150):
        for d in df_train_sift_des[i]:
            train_x.append(d)
            train_y.append(df_train_class[i])
    return train_x, train_y

In [10]:
# Naive kNN
def test(classifier, split_id):
    df_test = df[df["split_" + str(split_id)] == "test"]
    df_test_sift_des = list(df_test["sift_kp_descriptors"])
    df_test_name = list(df_test["name"])
    predict = np.zeros(50, dtype=np.uint8)
    for i in range(0, 50):
        progress = int(i / 50.0 * 1000) / 10.0
        print("split_{} {} {}%".format(split_id, df_test_name[i], progress), end="\r")
        vote = np.zeros(50, dtype=np.uint16)
        for d in df_test_sift_des[i]:
            vote[classifier.predict(d)[0]] += 1
        predict[i] = vote.argmax()
    return predict

In [11]:
# Naive kNN
classifier = kNN(n_neighbors=5, weights="distance", metric=cos_dist)
tp_count = np.zeros(10, dtype=np.uint8)
for i in range(0, 10):
    train_x, train_y = get_training_set(i)
    predict_y = test(classifier.fit(train_x, train_y), i)
    tp_count[i] = sum(np.linspace(0, 49, num=50) == predict_y)
print(tp_count)
accuracy = tp_count.astype(np.float32) / 50
print("mean_accuracy={}, sigma={}".format(np.mean(accuracy), np.std(accuracy)))

[42 43 39 40 40 44 43 42 43 40]
mean_accuracy=0.832000076771, sigma=0.0324961580336
